In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Importing Data

The inputs and results are given as an input and target to neural network. It will be used for training the model. In this model, we define a coefficient to each job which we called a_i. With this coefficient, we reduce our jobs' dimensions 4 to 1. Then ML system try to learn relation between this a_i and the actual schedule. 

In [2]:
def load_data_from_file(filename):
    inputs, targets = [], []
    with open(filename, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 2):
            input_list = np.array([float(x) for x in lines[i].strip().split()], dtype=np.float32)
            inputs.append(input_list)
            
            target_list = np.array([int(x) for x in lines[i + 1].strip().split()], dtype=np.float32)
            targets.append(target_list)
            
    return np.array(inputs, dtype=np.float32), np.array(targets, dtype=np.float32)

file = 'data.txt'
inputs, targets = load_data_from_file(file)
inputs = np.array(inputs, dtype=np.float32)
targets = np.array(targets, dtype=np.float32)

# Defining The Network

Our Neural Network System is defined as a class with dimension 10.

In [3]:
class RankNet(nn.Module):
    def __init__(self):
        super(RankNet, self).__init__()
        self.hidden = nn.Linear(10, 50)
        self.output = nn.Linear(50, 10)
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = self.sigmoid(self.hidden(x))
        x = self.output(x) 
        return x

# Creating The Network

Our model is created as an object and its loss function and optimizer is created.

In [4]:
model = RankNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

inputs_tensor = torch.tensor(inputs)
targets_tensor = torch.tensor(targets)

if len(inputs_tensor.shape) == 1 or inputs_tensor.shape[1] != 10:
    inputs_tensor = inputs_tensor.view(-1, 10) 

# Training

Our model is trained(calculate the loss and reconfigure its coefficients).

In [5]:
epochs = 1000  
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(inputs_tensor)
    loss = criterion(outputs, targets_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        

Epoch [100/1000], Loss: 8.1370
Epoch [200/1000], Loss: 7.9350
Epoch [300/1000], Loss: 7.8177
Epoch [400/1000], Loss: 7.7432
Epoch [500/1000], Loss: 7.6779
Epoch [600/1000], Loss: 7.6130
Epoch [700/1000], Loss: 7.5502
Epoch [800/1000], Loss: 7.4999
Epoch [900/1000], Loss: 7.4600
Epoch [1000/1000], Loss: 7.4107


# Calculating Schedules For New Problem

The model will predict a schedule for new input set in line with its previous coefficents

In [6]:
test_input = torch.tensor([6.00,6.00,4.20,2.50,21.00,4.40,6.00,5.75,5.67,13.00])
model.eval()
with torch.no_grad():
    test_output = model(test_input)

print("Test Input:", test_input)
print("Predicted Ranks (as indices):", test_output.flatten().numpy().argsort() + 1)
print(test_output)

Test Input: tensor([ 6.0000,  6.0000,  4.2000,  2.5000, 21.0000,  4.4000,  6.0000,  5.7500,
         5.6700, 13.0000])
Predicted Ranks (as indices): [ 6  9  5  2  1  8  4  7  3 10]
tensor([5.3943, 5.2011, 5.8840, 5.6313, 5.1819, 5.0427, 5.6822, 5.5456, 5.1434,
        6.2217])
